In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers, callbacks, preprocessing, utils
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

True

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3096)])

In [5]:
path_to_raw_data = "../data/raw/digit-recognizer/"

In [6]:
df_raw_train = pd.read_csv(path_to_raw_data + "train.csv").fillna(0)
df_raw_test = pd.read_csv(path_to_raw_data + "test.csv").fillna(0)

In [7]:
df_raw_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_raw_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
feature_cols = ["pixel{i}".format(i=i) for i in range(784)]
target_col = "label"

In [10]:
x_train = df_raw_train[feature_cols].values.reshape(-1, 28, 28, 1)
y_train = utils.to_categorical(df_raw_train[[target_col]].values.flatten())

x_test = df_raw_test[feature_cols].values.reshape(-1, 28, 28, 1)

In [11]:
# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

In [12]:
print(np.shape(x_train))

(42000, 28, 28, 1)


In [13]:
x_train_train, x_train_eval, y_train_train, y_train_eval = train_test_split(x_train, y_train, test_size=0.33, random_state=42)

In [14]:
# Training Parameters
learning_rate = 0.001
num_steps = 2000
batch_size = 128

# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.25 # Dropout, probability to drop a unit


In [15]:
model = models.Sequential()

model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Dropout(0.25))


model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation = "softmax"))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0

In [17]:
# Define the optimizer
optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [18]:
# Set a learning rate annealer
learning_rate_reduction = callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [19]:
epochs = 300 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86

In [20]:
datagen = preprocessing.image.ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [21]:
datagen.fit(x_train_train)

In [22]:
# history = model.fit(x_train_train, y_train_train, epochs=10, 
#                     validation_data=(x_train_eval, y_train_eval))

history = model.fit_generator(datagen.flow(x_train_train, y_train_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_train_eval, y_train_eval),
                              verbose = 2, steps_per_epoch=x_train_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Epoch 1/300
327/327 - 20s - loss: 0.5014 - accuracy: 0.8404 - val_loss: 0.0739 - val_accuracy: 0.9784
Epoch 2/300
327/327 - 19s - loss: 0.1554 - accuracy: 0.9538 - val_loss: 0.0453 - val_accuracy: 0.9859
Epoch 3/300
327/327 - 20s - loss: 0.1099 - accuracy: 0.9670 - val_loss: 0.0378 - val_accuracy: 0.9884
Epoch 4/300
327/327 - 19s - loss: 0.0897 - accuracy: 0.9738 - val_loss: 0.0383 - val_accuracy: 0.9881
Epoch 5/300
327/327 - 19s - loss: 0.0796 - accuracy: 0.9763 - val_loss: 0.0275 - val_accuracy: 0.9913
Epoch 6/300
327/327 - 19s - loss: 0.0760 - accuracy: 0.9785 - val_loss: 0.0277 - val_accuracy: 0.9914
Epoch 7/300
327/327 - 19s - loss: 0.0692 - accuracy: 0.9796 - val_loss: 0.0289 - val_accuracy: 0.9908
Epoch 8/300
327/327 - 19s - loss: 0.0614 - accuracy: 0.9819 - val_loss: 0.0264 - val_accuracy: 0.9921
Epoch 9/300
327/327 - 19s - loss: 0.0629 - accuracy: 0.9820 - val_loss: 0.0251 - val_accuracy: 0.9928
Epoch 10/300
327/327 - 19s - loss: 0.0610 - accuracy: 0.9827 - val_loss: 0.0225 - 

In [23]:
np.shape(x_test)

(28000, 28, 28, 1)

In [24]:
y_test = model.predict_classes(tf.cast(x_test, tf.float32))

In [25]:
df_prediction = pd.DataFrame({'ImageId' : df_raw_test.index + 1, 'Label' : y_test})

In [26]:
df_prediction.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [27]:
df_prediction.to_csv("../data/processed/digit-recognizer/result_cnn_3.csv", index=False)